<a href="https://colab.research.google.com/github/siddharth-iyer1/460J-Labs/blob/main/DSL_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Lab: Lab 5

Submit:
1. A pdf of your notebook with solutions.
2. A link to your colab notebook or also upload your .ipynb if not working on colab.

# Goals of this Lab

1. Random Forests
2. Boosting
3. Playing with Ensembling packages, including XGBoost and CatBoost
4. One more time: Revisiting CIFAR-10 and MNIST
5. Getting ready for Kaggle

We will soon open a Kaggle competition made for this class. In that one, you will be participating on your own. This is an intro to get us started, and also an excuse to work with regularization and regression which we have been discussing. You'll revisit some problems from earlier labs, this time using Random Forests, and Boosting. In particular, you should take this opportunity to become familiar with some very useful packages for boosting. I recommend not only the boosting packages in scikit-learn, but also XGBoost, GBM Light, CatBoost and possibly others. You have to download these and get them running, and then read their documentation to figure out how they work, what the hyperparameters are, etc.

Also, the metric we will use in the Kaggle competition is AUC. We will discuss this. In the meantime, you may want to understand how it works. At least one key thing to remember: to get a good AUC score, you need to submit a soft score (probabilities) and not rounded values (i.e., not 0s and 1s).


## Problem 1: Revisiting Logistic Regression and MNIST

We have played with the handwriting recognition problem (the MNIST data set) using decision trees. We have also considered the same problem using multi-class Logistic Regression in a previous Lab. We revisit this one more time.

**Part 1**: Use Random Forests to try to get the best possible *test accuracy* on MNIST. This involves getting acquainted with how Random Forests work, understanding their parameters, and therefore using Cross Validation to find the best settings. How well can you do? You should use the accuracy metric, since this is what you used in the previous Lab  -- therefore this will allow you to compare your results from Random Forests with your results from L1- and L2- Regularized Logistic Regression.

What are the hyperparameters of your best model?

**Part 2**: Use Boosting to do the same. Take the time to understand how XGBoost works (and/or other boosting packages available -- CatBoost is also another favorite). Try your best to tune your hyper-parameters. As added motivation: typically the winners and near-winners of the Kaggle competition are those that are best able to tune and cross validate XGBoost. What are the hyperparameters of your best model?


In [2]:
# MNIST Data Download
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [6]:
# MNIST Random Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Global Hyperparameters
TEST_SIZE = 0.25
NUM_ESTIMATORS = 100

# Setup train test split
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=TEST_SIZE, random_state=42)

# Baseline Random Forest
clf = RandomForestClassifier(n_estimators=NUM_ESTIMATORS, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9653142857142857


In [8]:
# Run Cross Validation to Optimize Hyperparameters
from sklearn.model_selection import cross_val_score
import numpy as np

# Cross Validation
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
print(f"Cross Validation Scores: {scores}")
print(f"Mean: {np.mean(scores)}")
print(f"Standard Deviation: {np.std(scores)}")

Cross Validation Scores: [0.966      0.96628571 0.96809524 0.96933333 0.964     ]
Mean: 0.9667428571428573
Standard Deviation: 0.0018343163721910364


In [13]:
# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV

# set bootstrap to false because in all other cases, when changing other hyperparams, bootstrap was false
param_grid = [
    {'n_estimators': [100, 150, 200], 'max_features': [6, 7, 8, 10], 'bootstrap': [False]}
]

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring="accuracy", return_train_score=True)

In [16]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid=[{'bootstrap': [False], 'max_features': [6, 8, 10],
                          'n_estimators': [100, 150, 200]}],
             return_train_score=True, scoring='accuracy')

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

predictions = grid_search.predict(X_test)

best_params = grid_search.best_params_
best_scores = grid_search.best_score_

print("Best parameters found by grid search:", best_params)
print("Best scores found by grid search:", best_scores)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Calculate precision
# Note: For multiclass classification, you need to set the 'average' parameter.
# Example: average='macro' calculates metrics for each label, and finds their unweighted mean. It does not take label imbalance into account.
# For binary classification, you can omit the 'average' parameter or set it to 'binary' (default).
precision = precision_score(y_test, predictions, average='macro')  # Change 'macro' as needed
print(f"Precision: {precision}")

recall = recall_score(y_test, predictions, average='macro')  # Change 'macro' as needed
print(f"Recall: {recall}")

Best parameters found by grid search: {'bootstrap': False, 'max_features': 10, 'n_estimators': 200}
Best scores found by grid search: 0.9696380952380952
Accuracy: 0.9689142857142857
Precision: 0.9687101334449796
Recall: 0.9685573192546881


In [ ]:
%pip install xgboost

In [11]:
y_train_boost = y_train.astype(int)
y_test_boost = y_test.astype(int)

In [26]:
#Part 2
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

param_grid = {
    'n_estimators': [200],
    'max_depth': [6,7],
    'learning_rate': [0.1]
}

# Create the XGBoost model object
xgb_model = xgb.XGBClassifier()

# Create the GridSearchCV object
grid_search2 = GridSearchCV(xgb_model, param_grid, cv=2, scoring='accuracy', return_train_score=True)

# Fit the GridSearchCV object to the training data
grid_search2.fit(X_train, y_train_boost)





GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
           

In [27]:
predictions2 = grid_search2.predict(X_test)

best_params2 = grid_search2.best_params_
best_scores2 = grid_search2.best_score_


print("Best parameters found by grid search:", best_params2)
print("Best scores found by grid search:", best_scores2)

accuracy = accuracy_score(y_test_boost, predictions2)
print(f"Accuracy: {accuracy}")

Best parameters found by grid search: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
Best scores found by grid search: 0.9691809523809525
Accuracy: 0.9745714285714285


## Problem 2: Revisiting Logistic Regression and CIFAR-10

Now that you have your pipeline set up, it should be easy to apply the above procedure to CIFAR-10. If you did something that takes significant computation time, keep in mind that CIFAR-10 is a few times larger.

**Part 1**: What is the best accuracy you can get on the test data, by tuning Random Forests? What are the hyperparameters of your best model?

**Part 2**: What is the best accuracy you can get on the test data, by tuning XGBoost? What are the hyperparameters of your best model?

In [2]:
# Part 1

import numpy as np
from keras.datasets import cifar10
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Sample a smaller subset for faster execution
# Adjust the test_size to change the subset size, e.g., 0.02 for 2%
X_train_sub, _, y_train_sub, _ = train_test_split(X_train, y_train, test_size=0.98, random_state=42)
X_test_sub, _, y_test_sub, _ = train_test_split(X_test, y_test, test_size=0.98, random_state=42)

# Flatten the images
X_train_flat = X_train_sub.reshape(X_train_sub.shape[0], -1)
X_test_flat = X_test_sub.reshape(X_test_sub.shape[0], -1)

# Flatten the labels
y_train_flat = y_train_sub.ravel()
y_test_flat = y_test_sub.ravel()

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_flat, y_train_flat)

y_pred = clf.predict(X_test_flat)

accuracy = accuracy_score(y_test_flat, y_pred)
print(f"Baseline RandomForest Accuracy on subset: {accuracy}")


Baseline RandomForest Accuracy on subset: 0.31


In [3]:
# TUNING on the subset

param_grid = {
    'n_estimators': [50, 100],  
    'max_depth': [None, 10],  
    'min_samples_split': [2, 4],  
}

clf = RandomForestClassifier(random_state=42)

# Setup GridSearchCV with fewer CV folds and adjusted parameters
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train_flat, y_train_flat)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_flat)
test_accuracy = accuracy_score(y_test_flat, y_pred)
print(f"Test set accuracy of the best model on subset: {test_accuracy}")


Fitting 2 folds for each of 8 candidates, totalling 16 fits
Best parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.312
Test set accuracy of the best model on subset: 0.31


In [4]:
# Part 2

import xgboost as xgb
from sklearn.metrics import accuracy_score

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Fit the classifier to the training set
xgb_clf.fit(X_train_flat, y_train_flat)

# Predict on the test set
y_pred_xgb = xgb_clf.predict(X_test_flat)

# Calculate accuracy
accuracy_xgb = accuracy_score(y_test_flat, y_pred_xgb)
print(f"Baseline XGBoost Accuracy: {accuracy_xgb}")


Baseline XGBoost Accuracy: 0.32


In [5]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid_xgb = {
    'n_estimators': [50, 100],  # Number of trees
    'learning_rate': [0.1, 0.01],  # Step size shrinkage used to prevent overfitting
    'max_depth': [6, 10],  # Maximum depth of a tree
    'subsample': [0.8, 1],  # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1],  # Subsample ratio of columns when constructing each tree
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Setup GridSearchCV
grid_search_xgb = GridSearchCV(estimator=xgb_clf, param_grid=param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search_xgb.fit(X_train_flat, y_train_flat)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search_xgb.best_params_}")
print(f"Best cross-validation score: {grid_search_xgb.best_score_}")

# Evaluate on the test set
best_model_xgb = grid_search_xgb.best_estimator_
y_pred_best_xgb = best_model_xgb.predict(X_test_flat)
test_accuracy_best_xgb = accuracy_score(y_test_flat, y_pred_best_xgb)
print(f"Test set accuracy of the best XGBoost model: {test_accuracy_best_xgb}")


Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best parameters: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.8}
Best cross-validation score: 0.32199265133396876
Test set accuracy of the best XGBoost model: 0.3


## Problem 3: Revisiting Kaggle

This is a continuation of Problem 2 from Lab 3. You already did some first steps there, including making a Kaggle account, and trying ridge and lasso linear regression. You also tried stacking.

**Part 1** (Nothing to hand in) Revisit Lab 3 and your answers there.

**Part 2**: Train a gradient boosting regression, e.g., using XGBoost. What score can you get just from a single XGB? (you will need to optimize over its parameters).

**Part 3**: Do your best to get a more accurate model. Try feature engineering and stacking many models. You are allowed to use any public tool in python. No non-python tools allowed.

**Part 4**: (Optional)  Read the Kaggle forums, tutorials and Kernels in this competition. This is an excellent way to learn. Include in your report if you find something in the forums you like, or if you made your own post or code post, especially if other Kagglers liked or used it afterwards.

**Other**: Be sure to read and learn the rules of Kaggle! No sharing of code or data outside the Kaggle forums. Every student should have their own individual Kaggle account and teams can be formed in the Kaggle submissions with your Lab partner. This is more important for live competitions of course.

In the real in-class Kaggle competition (which will be next), you will be graded based on your public score (include that in your report) and also on the creativity of your solution. In your report, due after the competition closes, you will explain what worked and what did not work. Many creative things will not work, but you will get partial credit for developing them. You can start thinking about this now.

In [66]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import skew
from scipy.stats.stats import pearsonr


# Load the training and testing datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Extract and remove the 'Id' column from the test dataset
ids = test_df.pop('Id')

# Combine training and testing data for consistent preprocessing
all_data = pd.concat((train_df.loc[:, 'MSSubClass':'SaleCondition'],
                      test_df.loc[:, 'MSSubClass':'SaleCondition']))

# Set the figure size for matplotlib plots

# Plot the distribution of the SalePrice and its logarithm to check skewness

# Apply log transformation to the SalePrice to reduce skewness
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

# Identify numeric features from all data
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Compute skewness of numeric features and apply log transformation to skewed features
skewed_feats = train_df[numeric_feats].apply(lambda x: skew(x.dropna()))  # Compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]  # Identify features with skewness > 0.75
skewed_feats = skewed_feats.index  # Get the names of skewed features

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])  # Apply log1p transformation to skewed features

# Convert categorical variables into dummy/indicator variables and fill missing values with column mean
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())

# Split the combined data back into training and testing datasets
X_train = all_data[:train_df.shape[0]]
X_test = all_data[train_df.shape[0]:]
y = train_df.SalePrice  # Target variable for the training data

X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=TEST_SIZE, random_state=42)


In [96]:
#Part 2
xgb_model_kaggle = xgb.XGBRegressor()
param_grid = {
    'n_estimators': [200,300],
    'max_depth': [7,10,15],
    'learning_rate': [.01, 0.1,.2]
}

grid_search_kaggle = GridSearchCV(xgb_model_kaggle, param_grid, cv=5)
grid_search_kaggle.fit(X_train,y_train)



GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
    

In [97]:
from sklearn.metrics import mean_squared_error

train_predict = grid_search_kaggle.predict(X_train)
test_predict = grid_search_kaggle.predict(X_test)
final_predictions = np.expm1(test_predict)  # Apply expm1 to invert the log1p transformation
mse_train = mean_squared_error(y_train,train_predict)
mse_test = mean_squared_error(y_test,test_predict)
print("MSE for train:" + str(mse_train))
print("MSE for test:" + str(mse_test))
#output = pd.DataFrame({'Id': ids, 'SalePrice': final_predictions.squeeze()})

#Got score on kaggle of .13754


MSE for train:5.913801238832761e-06
MSE for test:0.019767494333076403


In [103]:
#Part 3
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
best_boost = xgb.XGBRegressor(**grid_search_kaggle.best_params_)
best_boost.fit(X_train,y_train)

# Initialize additional regression models
rf_reg = RandomForestRegressor()

# Create a stack of models
stacked_reg = StackingRegressor(
    estimators=[('xgb', best_boost), ('rf', rf_reg)],
    final_estimator=xgb.XGBRegressor()
)

stacked_reg.fit(X_train, y_train)


StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_ra...
                                               grow_policy=None,
      

In [104]:
train_predict = stacked_reg.predict(X_train)
test_predict = stacked_reg.predict(X_test)
final_predictions = np.expm1(test_predict)  # Apply expm1 to invert the log1p transformation
mse_train = mean_squared_error(y_train,train_predict)
mse_test = mean_squared_error(y_test,test_predict)
print("MSE for train:" + str(mse_train))
print("MSE for test:" + str(mse_test))

MSE for train:0.012235727885252784
MSE for test:0.029993867462066205
